In [ ]:
import pandas
import re
from openbabel import pybel
import os
from opencadd.structure.core import Structure
from opencadd.io.dataframe import DataFrame
import warnings
import numpy as np
import subprocess
import tqdm

# filter warnings
warnings.filterwarnings("ignore")
ob_log_handler = pybel.ob.OBMessageHandler()
pybel.ob.obErrorLog.SetOutputLevel(0)

def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """
    Convert a PDB file to a PDBQT file needed by docking programs of the AutoDock family.

    Parameters
    ----------
    pdb_path: str or pathlib.Path
        Path to input PDB file.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    # add hydrogens at given pH
    #molecule.OBMol.CorrectForPH(pH)
    #molecule.addh()
    # add partial charges to each atom
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    pdbqt_path_new = os.path.join(os.getcwd(),os.path.basename(pdbqt_path))
    molecule.write("pdbqt", pdbqt_path_new, overwrite=True)
    return pdbqt_path_new

#%%
def smina_screen(receptor_fn, ligand_fn, pocket_fn, out_fn, log_fn, out_df, num_modes=20, exhaustiveness=8, cpu=12):
	# convert protein to PDBQT format
	pdbqt_path = pdb_to_pdbqt(receptor_fn, receptor_fn.rstrip('.pdb')+".pdbqt")

	#%%
	structure_df = DataFrame.from_file(pocket_fn)
	positions = np.array([structure_df["atom.y"].values,structure_df["atom.x"].values,structure_df["atom.z"].values])
	pocket_center = list(map(str,((np.max(positions,axis=1) + np.min(positions,axis=1)) / 2)))
	pocket_size = list(map(str,((np.max(positions,axis=1) - np.min(positions,axis=1)) + 5)))

	subprocess.call(f"smina -r {pdbqt_path} -l {ligand_fn} \
		--center_x {pocket_center[0]} --center_y {pocket_center[1]} --center_z {pocket_center[2]} \
		--size_x {pocket_size[0]} --size_y {pocket_size[1]} --size_z {pocket_size[2]} --out {out_fn} \
		--num_modes {num_modes} --exhaustiveness {exhaustiveness} --cpu {cpu} --log {log_fn}",shell=True)
	
	ligands = []
	with open(out_fn,"r") as smina_output:
		lines = smina_output.readlines()
		lines = [line.rstrip('\n') for line in lines]
		ligands.append(lines[0])
		read_flag = 0
		for line in lines:
	
			if read_flag:
				ligands.append(line)
	
			if not line.startswith("$$$$"):
				read_flag = 0
				continue
			else:
				read_flag = 1
				continue
	
	df_results = pandas.DataFrame(columns=["Pose","Score","RMSD_lb","RMSD_ub"])

	with open(log_fn,"r") as smina_output:
		lines = smina_output.readlines()
		lines = [line.rstrip('\n') for line in lines]
	
		read_flag = 0
		for line in lines:
	
			if line.startswith("Using random seed"):
				read_flag = 0
				continue
	
			if read_flag:
				line2 = line.split(' ')
				line2 = [el for el in line2 if el]
				line2 = list(map(float,line2))
				df_results.loc[len(df_results),:] = line2
	
			if line.startswith("-----"):
				read_flag = 1
				continue
	
	df_results['Ligand'] = ligands
	df_results['receptor'] = [receptor_fn]*len(df_results)
	df_results['ligand'] = [ligand_fn]*len(df_results)
	df_results['pocket'] = [pocket_fn]*len(df_results)
	df_results['out'] = [out_fn]*len(df_results)
	df_results['log'] = [log_fn]*len(df_results)
	df_results['num_modes'] = [num_modes]*len(df_results)
	df_results['exhaustiveness'] = [exhaustiveness]*len(df_results)
	df_results['cpu'] = [cpu]*len(df_results)
	df_results.to_csv(out_df, index=False)
	print(df_results)

In [ ]:
cluster_pockets = [f for f in os.listdir("../data/cluster_pockets/") if f.endswith(".pdb") and "res" in f]
# Get full path of cluster pockets
cluster_pockets = [os.path.join("../data/cluster_pockets/",f) for f in cluster_pockets]

cluster_frames = [f for f in os.listdir("../data/cluster_pockets/") if f.endswith(".pdb") and not "res" in f and not "gps" in f]
# Get full path of cluster frames
cluster_frames = [os.path.join("../data/cluster_pockets/",f) for f in cluster_frames]

# Get full path of cluster ligands
cluster_ligands = [f for f in os.listdir("../data/") if f.endswith(".sdf") and f.startswith("cluster_")]
cluster_ligands = [os.path.join("../data/",f) for f in cluster_ligands]

In [ ]:
for (cluster_frame,cluster_pocket) in zip(cluster_frames,cluster_pockets):
    for cluster_ligand in tqdm.tqdm(cluster_ligands):
        repeats = np.arange(1,11)
        for rep in tqdm.tqdm(repeats):
        # pc: pocket cluster
        # lc: ligand cluster
            smina_screen(cluster_frame, cluster_ligand, cluster_pocket,
                                        os.path.basename(cluster_pocket).rstrip(".pdb")+"_"+
                                        os.path.basename(cluster_ligand).rstrip('.sdf')+"_"+str(rep)+".sdf",
                                        os.path.basename(cluster_pocket).rstrip(".pdb")+"_"+
                                        os.path.basename(cluster_ligand).rstrip('.sdf')+"_"+str(rep)+".log",
                                        os.path.basename(cluster_pocket).rstrip(".pdb")+"_"+
                                        os.path.basename(cluster_ligand).rstrip('.sdf')+"_"+str(rep)+".csv",
                                        num_modes=20, exhaustiveness=16, cpu=20)